# Asyncio Practice
Examples taken from https://docs.python.org/3/library/asyncio-task.html#awaitables

In [1]:
import asyncio
import sys
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

async def say_after(delay, what):
    await asyncio.sleep(delay)
    logger.debug(what)

async def main():
    logger.debug('Started')
    
    await say_after(1, 'hello')
    await say_after(2, 'world')
    
    logger.debug('Finished')

await main() 

2021-06-20 12:33:04 | MainThread | DEBUG    | root | Started
2021-06-20 12:33:05 | MainThread | DEBUG    | root | hello
2021-06-20 12:33:07 | MainThread | DEBUG    | root | world
2021-06-20 12:33:07 | MainThread | DEBUG    | root | Finished


In [2]:
## io-bound 
import asyncio
import sys
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks(detail=False):
    logger.info(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    if detail is True:
        for t in asyncio.all_tasks():
            logger.info(t)
            logger.info('===============================')

async def say_after(delay, what):
    logger.debug(f'Waiting {what} for {delay} sec')
    await asyncio.sleep(delay)
    logger.debug(what)

async def main():
    logger.debug('Started')
    print_all_tasks()
    
    task1 = asyncio.create_task(
        say_after(1, 'hello'))
    print_all_tasks()
    
    task2 = asyncio.create_task(
        say_after(2, 'world'))
    print_all_tasks()
    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    logger.debug('Waiting task1')
    await task1
    
    print_all_tasks()
    logger.debug('Waiting task2')
    await task2
    
    print_all_tasks()
    logger.debug('Finished')

await main()

2021-06-20 12:33:07 | MainThread | DEBUG    | root | Started
2021-06-20 12:33:07 | MainThread | INFO     | root | ========== 1 Tasks ==========
2021-06-20 12:33:07 | MainThread | INFO     | root | ========== 2 Tasks ==========
2021-06-20 12:33:07 | MainThread | INFO     | root | ========== 3 Tasks ==========
2021-06-20 12:33:07 | MainThread | DEBUG    | root | Waiting task1
2021-06-20 12:33:07 | MainThread | DEBUG    | root | Waiting hello for 1 sec
2021-06-20 12:33:07 | MainThread | DEBUG    | root | Waiting world for 2 sec
2021-06-20 12:33:08 | MainThread | DEBUG    | root | hello
2021-06-20 12:33:08 | MainThread | INFO     | root | ========== 2 Tasks ==========
2021-06-20 12:33:08 | MainThread | DEBUG    | root | Waiting task2
2021-06-20 12:33:09 | MainThread | DEBUG    | root | world
2021-06-20 12:33:09 | MainThread | INFO     | root | ========== 1 Tasks ==========
2021-06-20 12:33:09 | MainThread | DEBUG    | root | Finished


## [Tasks](https://docs.python.org/3/library/asyncio-task.html#awaitables)

Tasks are used to schedule coroutines concurrently.

When a coroutine is wrapped into a Task with functions like asyncio.create_task() the coroutine is automatically scheduled to run soon.

When execute asyncio.create_task(), a task is added in the running event loop, when encounter keyword await, the current execution will be stopped and the control will be switched to the next process until the previous process is done.

In [3]:
# cpu-bound
import asyncio
import sys
import time
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks(detail=False):
    logger.info(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    if detail is True:
        for t in asyncio.all_tasks():
            logger.info(t)
            logger.info('===============================')

async def say_after(delay, what):
    logger.debug(f'Waiting {what} for {delay} sec')
    time.sleep(delay)
    logger.debug(what)

async def main():
    logger.debug('Started')
    print_all_tasks()
    
    task1 = asyncio.create_task(
        say_after(1, 'hello'))
    print_all_tasks()
    
    task2 = asyncio.create_task(
        say_after(2, 'world'))
    print_all_tasks()
    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    logger.debug('Waiting task1')
    await task1
    
    print_all_tasks()
    logger.debug('Waiting task2')
    await task2
    
    print_all_tasks()
    logger.debug('Finished')

await main()

2021-06-20 12:33:09 | MainThread | DEBUG    | root | Started
2021-06-20 12:33:09 | MainThread | INFO     | root | ========== 1 Tasks ==========
2021-06-20 12:33:09 | MainThread | INFO     | root | ========== 2 Tasks ==========
2021-06-20 12:33:09 | MainThread | INFO     | root | ========== 3 Tasks ==========
2021-06-20 12:33:09 | MainThread | DEBUG    | root | Waiting task1
2021-06-20 12:33:09 | MainThread | DEBUG    | root | Waiting hello for 1 sec
2021-06-20 12:33:10 | MainThread | DEBUG    | root | hello
2021-06-20 12:33:10 | MainThread | DEBUG    | root | Waiting world for 2 sec
2021-06-20 12:33:12 | MainThread | DEBUG    | root | world
2021-06-20 12:33:12 | MainThread | INFO     | root | ========== 1 Tasks ==========
2021-06-20 12:33:12 | MainThread | DEBUG    | root | Waiting task2
2021-06-20 12:33:12 | MainThread | INFO     | root | ========== 1 Tasks ==========
2021-06-20 12:33:12 | MainThread | DEBUG    | root | Finished


Coroutine is blocked by time.sleep() which is a simulated cpu-bound execution

In [4]:
# gather 1
import asyncio
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        logger.debug(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    logger.debug(f"Task {name}: factorial({number}) = {f}")
    return f

async def main():
    # Schedule three calls *concurrently*:
    L = asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    logger.debug(L)
    await L
    logger.debug(L)

await main()

2021-06-20 12:33:12 | MainThread | DEBUG    | root | <_GatheringFuture pending>
2021-06-20 12:33:12 | MainThread | DEBUG    | root | Task A: Compute factorial(2), currently i=2...
2021-06-20 12:33:12 | MainThread | DEBUG    | root | Task B: Compute factorial(3), currently i=2...
2021-06-20 12:33:12 | MainThread | DEBUG    | root | Task C: Compute factorial(4), currently i=2...
2021-06-20 12:33:13 | MainThread | DEBUG    | root | Task A: factorial(2) = 2
2021-06-20 12:33:13 | MainThread | DEBUG    | root | Task B: Compute factorial(3), currently i=3...
2021-06-20 12:33:13 | MainThread | DEBUG    | root | Task C: Compute factorial(4), currently i=3...
2021-06-20 12:33:14 | MainThread | DEBUG    | root | Task B: factorial(3) = 6
2021-06-20 12:33:14 | MainThread | DEBUG    | root | Task C: Compute factorial(4), currently i=4...
2021-06-20 12:33:15 | MainThread | DEBUG    | root | Task C: factorial(4) = 24
2021-06-20 12:33:15 | MainThread | DEBUG    | root | <_GatheringFuture finished resul

In [5]:
# gather 2
import asyncio
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks(detail=False):
    logger.info(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    if detail is True:
        for t in asyncio.all_tasks():
            logger.info(t)
            logger.info('===============================')

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        logger.debug(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    logger.debug(f"Task {name}: factorial({number}) = {f}")
    logger.info(f"Task {name} is finished.") # Expect CancelledError
    print_all_tasks()
    return f

async def main():
    logger.debug('Started scheduling')
    print_all_tasks()
    task1 = asyncio.create_task(factorial("A", 2))
    task2 = asyncio.create_task(factorial("B", 3))
    task3 = asyncio.create_task(factorial("C", 4))
    logger.debug('Finished scheduling')
    print_all_tasks()
    L = await asyncio.gather(task1, task2, task3)
    print_all_tasks()
    logger.debug(L)

await main()

2021-06-20 12:33:15 | MainThread | DEBUG    | root | Started scheduling
2021-06-20 12:33:15 | MainThread | INFO     | root | ========== 1 Tasks ==========
2021-06-20 12:33:15 | MainThread | DEBUG    | root | Finished scheduling
2021-06-20 12:33:15 | MainThread | INFO     | root | ========== 4 Tasks ==========
2021-06-20 12:33:15 | MainThread | DEBUG    | root | Task A: Compute factorial(2), currently i=2...
2021-06-20 12:33:15 | MainThread | DEBUG    | root | Task B: Compute factorial(3), currently i=2...
2021-06-20 12:33:15 | MainThread | DEBUG    | root | Task C: Compute factorial(4), currently i=2...
2021-06-20 12:33:16 | MainThread | DEBUG    | root | Task A: factorial(2) = 2
2021-06-20 12:33:16 | MainThread | INFO     | root | Task A is finished.
2021-06-20 12:33:16 | MainThread | INFO     | root | ========== 4 Tasks ==========
2021-06-20 12:33:16 | MainThread | DEBUG    | root | Task B: Compute factorial(3), currently i=3...
2021-06-20 12:33:16 | MainThread | DEBUG    | root | Ta

*awaitable* asyncio.**gather**(*aws, loop=None, return_exceptions=False)  
- Run awaitable objects in the aws sequence concurrently.  
- If any awaitable in aws is a coroutine, it is automatically scheduled as a Task.

Above two examples show that asyncio.**gather**() function accepts both coroutine functions or tasks.

When corountine functions return, that specific task is removed automatically.

In [6]:
# cancel gather 1
import asyncio
from asyncio import CancelledError
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks(detail=False):
    logger.info(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    if detail is True:
        for t in asyncio.all_tasks():
            logger.info(t)
            logger.info('===============================')

async def factorial(name, number):
    f = 1
    global L
    for i in range(2, number + 1):
        logger.debug(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    logger.debug(f"Task {name}: factorial({number}) = {f}")
    
    # Here, a task not really finish because it still not return yet
    logger.info(f"Task {name} is finished. Trying to cancel remaining tasks") # Expect CancelledError
    
    print_all_tasks(detail=True)
    
    # Here, because the tasks still not return, it turns out all 3 tasks are cancelled.
    L.cancel()
    return f

L = None
async def main():
    global L
    # Schedule three calls *concurrently*:
    L = asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    logger.debug(L)
    try:
        await L
    except CancelledError as ce:
        logger.warning(L)
    
    print_all_tasks(detail=True)

await main()

2021-06-20 12:33:18 | MainThread | DEBUG    | root | <_GatheringFuture pending>
2021-06-20 12:33:18 | MainThread | DEBUG    | root | Task A: Compute factorial(2), currently i=2...
2021-06-20 12:33:18 | MainThread | DEBUG    | root | Task B: Compute factorial(3), currently i=2...
2021-06-20 12:33:18 | MainThread | DEBUG    | root | Task C: Compute factorial(4), currently i=2...
2021-06-20 12:33:19 | MainThread | DEBUG    | root | Task A: factorial(2) = 2
2021-06-20 12:33:19 | MainThread | INFO     | root | Task A is finished. Trying to cancel remaining tasks
2021-06-20 12:33:19 | MainThread | INFO     | root | ========== 4 Tasks ==========
2021-06-20 12:33:19 | MainThread | INFO     | root | <Task pending name='Task-19' coro=<factorial() running at <ipython-input-6-5b1ced6066a2>:19> wait_for=<Future finished result=None> cb=[gather.<locals>._done_callback() at /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/asyncio/tasks.py:766]>
2021-06-20 12:33:19 | MainThread | INFO  

In this example, **Task A** is actually not finished because **Task A** is not returned yet. All 3 tasks including **Task A** are cancelled.

[Some **important** issue about asyncio.**gather**](https://stackoverflow.com/a/59074112)

In [7]:
# cancel gather 2
import asyncio
from asyncio import CancelledError
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks(detail=False):
    logger.info(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    if detail is True:
        for t in asyncio.all_tasks():
            logger.info(t)
            logger.info('===============================')

async def cancel_tasks():
    for task in asyncio.all_tasks():
        task.cancel()

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        logger.debug(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    logger.debug(f"Task {name}: factorial({number}) = {f}")
    print_all_tasks()
    return f

async def main():
    print_all_tasks()
    task1 = asyncio.create_task(factorial("A", 2))
    task0 = asyncio.create_task(cancel_tasks())
    task2 = asyncio.create_task(factorial("B", 3))
    task3 = asyncio.create_task(factorial("C", 4))
    print_all_tasks()
    L = asyncio.gather(task1, task0, task2, task3)
    try:
        await L
        logger.debug(L)
    except CancelledError as ce:
        logger.debug(L)
        logger.info('Remaining tasks are cancelled')

await main()

2021-06-20 12:33:19 | MainThread | INFO     | root | ========== 1 Tasks ==========
2021-06-20 12:33:19 | MainThread | INFO     | root | ========== 5 Tasks ==========
2021-06-20 12:33:19 | MainThread | DEBUG    | root | Task A: Compute factorial(2), currently i=2...
2021-06-20 12:33:19 | MainThread | DEBUG    | root | <_GatheringFuture finished exception=CancelledError()>
2021-06-20 12:33:19 | MainThread | INFO     | root | Remaining tasks are cancelled


In this example, unlike previous example (**Task A** is at least finished computational step and waiting for return), no task finished computational step because when **Task A** *await*, the control is switched to another execution which is cancel_tasks() function.

In [8]:
# cancel gather 2
import asyncio
from asyncio import CancelledError
sys.path.append('/Users/jack/Documents/Concurrency')
from asyncio_practice.setup_logger import logger

def print_all_tasks(detail=False):
    logger.info(f'========== {len(asyncio.all_tasks())} Tasks ==========')
    if detail is True:
        for t in asyncio.all_tasks():
            logger.info(t)
            logger.info('===============================')

def cancel_tasks(fut):
    logger.info(f'Received a future with value: {fut.result()}')
    logger.info('Remaining Tasks')
    print_all_tasks()
    for task in asyncio.all_tasks():
        task.cancel()

async def factorial(name, number):
    global L
    logger.debug(L)
    f = 1
    for i in range(2, number + 1):
        logger.debug(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    logger.debug(f"Task {name}: factorial({number}) = {f}")
    return f

L = None
async def main():
    global L
    logger.debug('Started scheduling')
    print_all_tasks()
    
    task1 = asyncio.create_task(factorial("A", 2))
    task2 = asyncio.create_task(factorial("B", 3))
    task3 = asyncio.create_task(factorial("C", 4))
    task1.add_done_callback(cancel_tasks)
    
    logger.debug('Finished scheduling')
    print_all_tasks()

    L = asyncio.gather(task1, task2, task3)
    try:
        logger.debug(L)
        await L
    except CancelledError as ce:
        L.done()
        logger.debug(L)
        logger.info('Remaining tasks are cancled')
    logger.debug(L)

await main()

2021-06-20 12:33:19 | MainThread | DEBUG    | root | Started scheduling
2021-06-20 12:33:19 | MainThread | INFO     | root | ========== 1 Tasks ==========
2021-06-20 12:33:19 | MainThread | DEBUG    | root | Finished scheduling
2021-06-20 12:33:19 | MainThread | INFO     | root | ========== 4 Tasks ==========
2021-06-20 12:33:19 | MainThread | DEBUG    | root | <_GatheringFuture pending>
2021-06-20 12:33:19 | MainThread | DEBUG    | root | <_GatheringFuture pending cb=[<TaskWakeupMethWrapper object at 0x7ffa07861b80>()]>
2021-06-20 12:33:19 | MainThread | DEBUG    | root | Task A: Compute factorial(2), currently i=2...
2021-06-20 12:33:19 | MainThread | DEBUG    | root | <_GatheringFuture pending cb=[<TaskWakeupMethWrapper object at 0x7ffa07861b80>()]>
2021-06-20 12:33:19 | MainThread | DEBUG    | root | Task B: Compute factorial(3), currently i=2...
2021-06-20 12:33:19 | MainThread | DEBUG    | root | <_GatheringFuture pending cb=[<TaskWakeupMethWrapper object at 0x7ffa07861b80>()]>
2